# **トランスポーター配列と基質のペアデータセットを構築するノートブック**
(参考) [SPOT論文](https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3002807) [ソースコード](https://github.com/AlexanderKroll/SPOT/blob/main/code/preprocessing/A3%20-%20Extract%20Transporter-substrate%20pairs%20from%20UniProt.ipynb)

---

## **0. 目的**

このノートブックでは、UniProt、Rhea、ChEBIのデータを統合し、トランスポータータンパク質配列とその基質のペアデータセットを構築します。
具体的な手順は以下の通りです。

-  **UniProtデータの抽出:**
    *   UniProtデータベースから、特定の条件（長さ、フラグメントの有無、GOアノテーション、触媒活性）を満たすタンパク質配列を抽出します。
-  **基質の特定:**
    *   Rheaデータベースから、Rhea IDとChEBI IDの対応関係を取得します。
    *   ChEBIデータベースから、ChEBI IDと基質名の対応関係を取得します。
-  **タンパク質配列と基質の対応付け:**
    *   UniProtデータとRheaデータをRhea IDで結合します。
    *   RheaデータとChEBIデータをChEBI IDで結合します。
    *   結果として得られたデータから、タンパク質配列と基質名のペアを作成します。
-  **データセットの作成と保存:**
    *   データセットをトレーニング用とテスト用に分割します。
    *   各データセットをJSON形式で保存し、モデルの学習に利用可能な形にします。

---

## **1. データの取得**

解析に必要なデータは以下のリンクから取得します。

-   **RheaとChEBI IDの対応表** (`rhea-reactions.txt`)
    -   [Rhea FTP](https://ftp.expasy.org/databases/rhea/txt/)

-   **ChEBI IDとChEBI名の対応表** (`compounds.tsv`)
    -   [ChEBI FTP](https://ftp.ebi.ac.uk/pub/databases/chebi/Flat_file_tab_delimited/)

-   **UniProtから特定条件を満たすデータセット** (`uniprotkb_length_50_TO_1024_AND_fragmen_2025_03_04.tsv`)
    -  **条件**
        -   長さ：50〜1024
        -   フラグメント：なし
        -   GOアノテーション：`go:0005215`（トランスポーター活性）
        -   触媒活性：含む
    -   クエリ：
        ```
        (length:[50 TO 1024]) AND (fragment:false) AND (go:0005215) AND (cc_catalytic_activity:*)
        ```
    -   **UniProtでクエリを実行してtsv形式でダウンロード**
        ![alt text](image.png)

**データ構造**

ダウンロードした3つのファイルは`dataset`ディレクトリに配置してください。

```bash
dataset/
    ├── compounds.tsv  # ChEBIデータ
    ├── rhea-reactions.txt  # Rheaデータ
    └── uniprotkb_length_50_TO_1024_AND_fragmen_2025_03_04.tsv  # UniProtデータ
```

---

## **2. 必要なライブラリのインポート**

In [43]:
import pandas as pd
import numpy as np
from os.path import join
import subprocess
import matplotlib.pyplot as plt
import warnings

# 警告を無視
warnings.filterwarnings('ignore')

---


## **3. UniProtデータの処理**

### **(1) UniProtデータの読み込み**

-   タブ区切り（TSV）で保存されているUniProtデータを読み込みます。

In [51]:
df_uniprot = pd.read_csv('dataset/uniprotkb_length_50_TO_1024_AND_fragmen_2025_03_04.tsv', sep='\t')
# df_uniprotを表示
df_uniprot

,Entry,Sequence,Catalytic activity,Gene Ontology (molecular function),Annotation
0,A0A015JYU2,MKNLPVSFSAESPGKVLFSPTSGISLFSLFPLFLFVPVFLLSFIIL...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type transporter activity [GO:0140359]; AT...,5.0
1,A0A024RA31,MASEFKKKLFWRAVVAEFLATTLFVFISIGSALGFKYPVGNNQTAV...,CATALYTIC ACTIVITY: Reaction=CO2(out) = CO2(in...,ephrin receptor binding [GO:0046875]; potassiu...,5.0
2,A0A034VPM6,MLYCPPNVTFSQIWINNGISHCFMDTIGNGVASGFMLLMGTVQLLM...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
3,A0A060X8Q2,MVVVQSYCEASASISEAWLEGGISPCFYFTLVPTVLLTLSFFFGAF...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
4,A0A060XRG5,MAENMEGKVSNGEAHLDEPVANGLRHKKCSNETTVEKIKRLVMANL...,CATALYTIC ACTIVITY: Reaction=D-serine(in) + L-...,antiporter activity [GO:0015297]; L-aspartate ...,5.0
...,...,...,...,...,...
1466198,X8JNV3,MSSTIHTPPTRPQNVLPESPGIQGRSSYIYSDTVQTRQDSLEISEK...,CATALYTIC ACTIVITY: Reaction=H2O(in) = H2O(out...,glycerol channel activity [GO:0015254]; water ...,1.0
1466199,X8JR86,MGSIDRRNVISAVFVGFASLGGFIMGFDTSVISGVKELPAWKQRFG...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0
1466200,X8JTZ6,MAAGWLADRIGRKRTIQAGSLVAILGCSLQTGAQNINFLLVGRVIA...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0
1466201,X8JVQ7,MGKGYSPRIAANPYIVGSFACIGGGLFGLDISSMSGVLSNPSYLRV...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0


### **(2) 標準アミノ酸のみのシーケンスにフィルタリング**

-   標準アミノ酸以外の文字が含まれている場合は除外されます。

In [52]:
standard_amino_acids = "ACDEFGHIKLMNPQRSTVWY" # 標準アミノ酸のパターンを定義
pattern = f"^[{standard_amino_acids}]+$"
df_uniprot = df_uniprot[df_uniprot['Sequence'].str.match(pattern)] # 標準アミノ酸のみを含むシーケンスを抽出
# df_uniprotを表示
df_uniprot

,Entry,Sequence,Catalytic activity,Gene Ontology (molecular function),Annotation
0,A0A015JYU2,MKNLPVSFSAESPGKVLFSPTSGISLFSLFPLFLFVPVFLLSFIIL...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type transporter activity [GO:0140359]; AT...,5.0
1,A0A024RA31,MASEFKKKLFWRAVVAEFLATTLFVFISIGSALGFKYPVGNNQTAV...,CATALYTIC ACTIVITY: Reaction=CO2(out) = CO2(in...,ephrin receptor binding [GO:0046875]; potassiu...,5.0
2,A0A034VPM6,MLYCPPNVTFSQIWINNGISHCFMDTIGNGVASGFMLLMGTVQLLM...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
3,A0A060X8Q2,MVVVQSYCEASASISEAWLEGGISPCFYFTLVPTVLLTLSFFFGAF...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
4,A0A060XRG5,MAENMEGKVSNGEAHLDEPVANGLRHKKCSNETTVEKIKRLVMANL...,CATALYTIC ACTIVITY: Reaction=D-serine(in) + L-...,antiporter activity [GO:0015297]; L-aspartate ...,5.0
...,...,...,...,...,...
1466198,X8JNV3,MSSTIHTPPTRPQNVLPESPGIQGRSSYIYSDTVQTRQDSLEISEK...,CATALYTIC ACTIVITY: Reaction=H2O(in) = H2O(out...,glycerol channel activity [GO:0015254]; water ...,1.0
1466199,X8JR86,MGSIDRRNVISAVFVGFASLGGFIMGFDTSVISGVKELPAWKQRFG...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0
1466200,X8JTZ6,MAAGWLADRIGRKRTIQAGSLVAILGCSLQTGAQNINFLLVGRVIA...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0
1466201,X8JVQ7,MGKGYSPRIAANPYIVGSFACIGGGLFGLDISSMSGVLSNPSYLRV...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0


### **(3) 配列のFASTA化 & 90%以上の類似性でクラスタリングし代表配列のみ抽出**

-  配列をFASTA形式で保存し、MMseqs2で90%以上の類似性でクラスタリングを実行します。
-  MMseqs2のクラスタリング結果から代表配列のみを抽出します。

In [ ]:
# 1. DataFrame から FASTA ファイルを生成
fasta_file = "dataset/uniprot_sequences.fasta"
with open(fasta_file, "w") as f:
    for _, row in df_uniprot.iterrows():
        f.write(f">{row['Entry']}\n{row['Sequence']}\n")

# 2. MMseqs2 でクラスタリングを実行
output_prefix = "dataset/clustered_uniprot"
tmp_dir = "tmp"

subprocess.run([
    "mmseqs", "easy-cluster", fasta_file, output_prefix, tmp_dir,
    "--min-seq-id", "0.9"
])

# 3. クラスタリング結果を解析
cluster_file = f"{output_prefix}_cluster.tsv"
clusters = pd.read_csv(cluster_file, sep="\t", names=["cluster_rep", "member"])


rep_sequences = clusters["cluster_rep"].unique() # 代表配列のリストを取得

# 4. DataFrame をフィルタリング
df_uniprot = df_uniprot[df_uniprot['Entry'].isin(rep_sequences)]

# df_uniqueを表示
df_uniprot

easy-cluster dataset/uniprot_sequences.fasta dataset/clustered_uniprot tmp --min-seq-id 0.9 

MMseqs Version:                     	13.45111
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Seed substitution matrix            	nucl:nucleotide.out,aa:VTML80.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	2147483647
Alphabet size                       	nucl:5,aa:21
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	0
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask lower case residues            	0
Minimum diagonal score              	15
Include identical seq.

CompletedProcess(args=['mmseqs', 'easy-cluster', 'dataset/uniprot_sequences.fasta', 'dataset/clustered_uniprot', 'tmp', '--min-seq-id', '0.9'], returncode=0)

,Entry,Sequence,Catalytic activity,Gene Ontology (molecular function),Annotation
5,A0A060XZ13,MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
7,A0A061HZY3,MSSSNDHVLVPMPQRNTNGLPGMNSSDTKTLTGDVLSFHHITYRVK...,CATALYTIC ACTIVITY: Reaction=17beta-estradiol ...,ABC-type xenobiotic transporter activity [GO:0...,5.0
9,A0A067QIY2,MLVKRFRISQIRMLYCPPNVTLGEVWINNGTSQCFMETLTVSITAG...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
12,A0A084W9E2,MIHYCPNDTSMDVIWFNHGVSQCFMDTVAMASIGGFMLLFGTLQLV...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
17,A0A091CQ10,MRSPWLPLGQWVTSELRLNTRVWSCDRHPQSLSRPTYGSISSSPHP...,CATALYTIC ACTIVITY: Reaction=Fe(2+)(in) + H(+)...,cadmium ion transmembrane transporter activity...,5.0
...,...,...,...,...,...
1466189,X8IR59,MKKERLIAFTDAVLAIIMTILVLELEKPDAPTLEAFWELRQNFFAY...,CATALYTIC ACTIVITY: Reaction=K(+)(in) = K(+)(o...,potassium channel activity [GO:0005267]; proto...,1.0
1466191,X8IV51,MAPAVAGAQAKVTLDPALLFKGRWWQHSHIFWLNILLLVPLVTSYA...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0
1466193,X8J197,MVPLGANFSPDDTPLDIVLYSGRWYQHRHLLILNMMLVIPLITSYA...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,carbohydrate:proton symporter activity [GO:000...,1.0
1466196,X8JM77,MTRPPSDTIAPTNYLSVAVGWVVVCAFQHGYHIAALNQISDVLSCK...,CATALYTIC ACTIVITY: Reaction=myo-inositol(out)...,hexose transmembrane transporter activity [GO:...,1.0


### **(?) Annotationスコアが3.0以上のものを抽出**

-   Annotationスコアはタンパク質の注釈の信頼度を表します。
-   信頼度の高いデータのみを対象にするため、スコアが3.0以上のものを抽出します。

In [48]:
df_uniprot = df_uniprot[df_uniprot['Annotation'] >= 3.0]

df_uniprot

,Entry,Sequence,Catalytic activity,Gene Ontology (molecular function),Annotation
5,A0A060XZ13,MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
7,A0A061HZY3,MSSSNDHVLVPMPQRNTNGLPGMNSSDTKTLTGDVLSFHHITYRVK...,CATALYTIC ACTIVITY: Reaction=17beta-estradiol ...,ABC-type xenobiotic transporter activity [GO:0...,5.0
9,A0A067QIY2,MLVKRFRISQIRMLYCPPNVTLGEVWINNGTSQCFMETLTVSITAG...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
12,A0A084W9E2,MIHYCPNDTSMDVIWFNHGVSQCFMDTVAMASIGGFMLLFGTLQLV...,CATALYTIC ACTIVITY: Reaction=coproporphyrin I(...,ABC-type heme transporter activity [GO:0015439...,5.0
17,A0A091CQ10,MRSPWLPLGQWVTSELRLNTRVWSCDRHPQSLSRPTYGSISSSPHP...,CATALYTIC ACTIVITY: Reaction=Fe(2+)(in) + H(+)...,cadmium ion transmembrane transporter activity...,5.0
...,...,...,...,...,...
520258,Z9JMX9,MAHASTKSLFTAELIVPAIGDAFKKLNPKELIRNPVMFVTACVALL...,CATALYTIC ACTIVITY: Reaction=K(+)(out) + ATP +...,ATP binding [GO:0005524]; ATP hydrolysis activ...,3.0
520259,Z9JTY6,MTTQETRKRSTAVVRPDEERPLGDIGARVRAGVAAWTRSPALDFYA...,CATALYTIC ACTIVITY: Reaction=[GlcNAc-(1->4)-Mu...,glycosyltransferase activity [GO:0016757]; lip...,3.0
520260,Z9JU65,MNPDTITLSAGSLTTVAVIVVIALIAVAMALRFRAEVLRAATGTPA...,CATALYTIC ACTIVITY: Reaction=diphosphate + H2O...,diphosphate hydrolysis-driven proton transmemb...,3.0
520261,Z9JXE3,MAELTIRPEDIRHALDTAVSTYQAGSTDREEIGRVTESADGIARVE...,CATALYTIC ACTIVITY: Reaction=ATP + H2O + 4 H(+...,ADP binding [GO:0043531]; ATP binding [GO:0005...,3.0


---

## **4. Rheaデータの処理**

### **(1) UniProt データから Rhea ID を抽出**

-   UniProtデータから、"Catalytic activity"列の情報を基に、対応するRhea IDを抽出します。
-   "(out)"と"(in)"を含む反応のみを対象とします。

In [32]:
start_rhea = "Xref=Rhea:"

def get_reaction_info(activity):
    # 最初のセミコロンまでの文字列を取得(反応名を取得)
    reaction = activity.split(";")[0]
    # "Reaction="を取り除く
    reaction = reaction.replace("Reaction=", "")

    # "Xref=Rhea:"の位置を探す
    start_pos = activity.find(start_rhea)
    if start_pos != -1:
        # Rhea IDを取得
        rhea_ID = activity[start_pos + len(start_rhea):]
        # 最初のカンマまでを取得
        rhea_ID = rhea_ID.split(",")[0]
    else:
        # 見つからない場合はNaNを返す
        rhea_ID = np.nan

    return(rhea_ID, reaction)

def get_activtities(activities):
    # "CATALYTIC ACTIVITY: "で分割
    activities = activities.split("CATALYTIC ACTIVITY: ")
    # 2番目以降の要素を取得
    return(activities[1:])

def process_row(row):
    # "Catalytic activity"列を取得
    activities = row["Catalytic activity"]
    # Nan出ないことを確認
    if pd.notna(activities):
        # get_activtities関数を呼び出し
        activities = get_activtities(activities=activities)

        # "(out)"と"(in)"を含む反応のみを処理
        return [
            (row["Entry"], reaction, rhea_ID)
            for activity in activities
            if "(out)" in activity and "(in)" in activity
            for rhea_ID, reaction in [get_reaction_info(activity)]
        ]
    return []

# 各行にprocess_row関数を適用し、結果をDataFrameに変換
df_reactions = pd.DataFrame(
    df_uniprot.apply(process_row, axis=1).explode().dropna().tolist(),
    columns=["Uniprot ID", "Reaction", "RHEA ID"]
)

# Number of different UniProt IDs : 356595
# Number of different RHEA reaction IDs: 1169

# df_reactionsを表示
df_reactions

,Uniprot ID,Reaction,RHEA ID
0,A0A060XZ13,coproporphyrin I(in) + ATP + H2O = coproporphy...,RHEA:66768
1,A0A060XZ13,coproporphyrin III(in) + ATP + H2O = coproporp...,RHEA:66664
2,A0A060XZ13,coproporphyrinogen III(in) + ATP + H2O = copro...,RHEA:66680
3,A0A060XZ13,heme b(in) + ATP + H2O = heme b(out) + ADP + p...,RHEA:19261
4,A0A060XZ13,pheophorbide a(in) + ATP + H2O = pheophorbide ...,RHEA:61360
...,...,...,...
212099,Y0KIE2,a quinone + NADH + 5 H(+)(in) = a quinol + NAD...,RHEA:57888
212100,Z9JMX9,K(+)(out) + ATP + H2O = K(+)(in) + ADP + phosp...,RHEA:16777
212101,Z9JU65,diphosphate + H2O + H(+)(in) = 2 phosphate + 2...,RHEA:13973
212102,Z9JXE3,ATP + H2O + 4 H(+)(in) = ADP + phosphate + 5 H...,RHEA:57720


### **(2) Rheaデータから反応情報(CHEBI ID)を抽出**

-   Rheaデータ（`rhea-reactions.txt`）を読み込み、各Rhea IDに対応するChEBI IDのリストを抽出します。
-   `(out)` または `(in)` を含む基質のみを対象とします。

In [33]:
def extract_RHEA_ID_and_CHEBI_IDs(entry):
    # ENTRYからRHEA IDを抽出
    RHEA_ID = entry[0][len("ENTRY"): -1]
    RHEA_ID = RHEA_ID.split(" ")[-1]
    # EQUATIONからCHEBI IDを抽出
    CHEBI_IDs = entry[2][len("EQUATION"): -1]
    CHEBI_IDs = CHEBI_IDs[CHEBI_IDs.index("CHEBI"):]

    # DEFINITIONから基質名を抽出
    sub_names = entry[1][len("DEFINITION"): -1]
    # 先頭に空白があれば削除
    while sub_names[0] == " ":
        sub_names = sub_names[1:]

    return(RHEA_ID, CHEBI_IDs, sub_names)

def get_substrate_IDs(IDs):
     # "="、"=>"、または"<=>"で分割し、最初の部分を取得
    IDs = IDs.split(" = ")[0]
    IDs = IDs.split(" => ")[0]
    IDs = IDs.split(" <=> ")[0]
    # "+"で分割し、";"で結合
    IDs = IDs.replace(" + ", ";")
    # ";"で分割
    IDs = IDs.split(";")
    # 各IDの最後の部分（CHEBI ID）を取得
    return([ID.split(" ")[-1] for ID in IDs])

def get_substrate_names(names):
    # "="、"=>"、または"<=>"で分割し、最初の部分を取得
    names = names.split(" = ")[0]
    names = names.split(" => ")[0]
    names = names.split(" <=> ")[0]
    # "+"で分割し、";"で結合
    names = names.replace(" + ", ";")
    # ";"で分割
    names = names.split(";")
     # 各IDの最後の部分（基質名）を取得
    return([ID.split(" ")[-1] for ID in names])

# データを収集するためのリスト
data_list = []

# ファイルをストリーミング形式で読み込み
with open('dataset/rhea-reactions.txt', 'r') as file1:
    entry_lines = []  # 1つのエントリを蓄積するリスト
    for line in file1:
        if line == '///\n':  # エントリの終わり
            if entry_lines:  # エントリが空でない場合
                # エントリを処理
                RHEA_ID, CHEBI_IDs, sub_names = extract_RHEA_ID_and_CHEBI_IDs(entry_lines)
                CHEBI_ID_list = get_substrate_IDs(IDs=CHEBI_IDs)
                sub_name_list = get_substrate_names(names=sub_names)

                # フィルタリング用のブール配列を作成
                access_array = [("(out)" in sub or "(in)" in sub) for sub in sub_name_list]
                # リスト内包表記でCHEBI_ID_listをフィルタリング
                CHEBI_ID_list = [chebi for chebi, access in zip(CHEBI_ID_list, access_array) if access]

                # RHEA IDを即座に数値に変換
                RHEA_ID_num = float(RHEA_ID.split(":")[-1])

                # データをリストに追加
                data_list.append({"RHEA ID": RHEA_ID_num, "CHEBI_ID_list": CHEBI_ID_list})

                entry_lines = []  # エントリをリセット
        else:
            entry_lines.append(line)  # エントリの行を蓄積

# データをデータフレームに変換
df_RHEA = pd.DataFrame(data_list)
# df_RHEAを表示
df_RHEA

,RHEA ID,CHEBI_ID_list
0,10000.0,[]
1,10001.0,[]
2,10002.0,[]
3,10003.0,[]
4,10004.0,[]
...,...,...
69683,82786.0,[]
69684,82787.0,[]
69685,82788.0,[]
69686,82789.0,[]


### **(3) Rheaデータを統合**

- `df_reactions`と`df_RHEA`を`RHEA ID`で結合
- 計算効率向上のため`lru_cache`を使用

In [34]:
from functools import lru_cache
from tqdm import tqdm

def get_CHEBI_IDs_from_RHEA_ID(rhea_ID):
    rhea_ID = float(rhea_ID.replace("RHEA:", ""))
    try:
        CHEBI_IDs = list(df_RHEA["CHEBI_ID_list"].loc[df_RHEA["RHEA ID"] == rhea_ID])[0]
    except IndexError:
        CHEBI_IDs = []
    return(CHEBI_IDs)

# 関数をキャッシュする
@lru_cache(maxsize=None)
def cached_get_CHEBI_IDs(rhea_id):
    return get_CHEBI_IDs_from_RHEA_ID(rhea_ID=rhea_id)

# より効率的なループバージョン
def optimized_loop():
    uniprot_ids = []
    chebi_ids = []
    
    # NaN値でないRHEA IDを持つ行だけを処理
    filtered_df = df_reactions.dropna(subset=["RHEA ID"])
    
    for _, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
        rhea_id = row["RHEA ID"]
        uid = row["Uniprot ID"]
        
        chebi_id_list = cached_get_CHEBI_IDs(rhea_id)
        
        # 各CHEBI IDに対してリストに追加
        for chebi_id in chebi_id_list:
            uniprot_ids.append(uid)
            chebi_ids.append(chebi_id)
    
    # 一度にデータフレームを作成
    df_prot_sub = pd.DataFrame({"Uniprot ID": uniprot_ids, "ChEBI": chebi_ids})
    df_prot_sub.drop_duplicates(inplace=True)
    return df_prot_sub

# 関数を実行
df_prot_sub = optimized_loop()
# df_prot_subを表示
df_prot_sub

100%|██████████| 212104/212104 [00:06<00:00, 31483.98it/s]


,Uniprot ID,ChEBI
0,A0A060XZ13,CHEBI:167478
1,A0A060XZ13,CHEBI:131725
2,A0A060XZ13,CHEBI:57309
3,A0A060XZ13,CHEBI:60344
4,A0A060XZ13,CHEBI:58687
...,...,...
243531,Y0KIE2,CHEBI:15378
243532,Z9JMX9,CHEBI:29103
243533,Z9JU65,CHEBI:15378
243534,Z9JXE3,CHEBI:15378


---

## **5. ChEBIデータの処理**

### **(1) ChEBI IDと基質名の対応表を読み込み**

-   ChEBI IDと基質名の対応表（`compounds.tsv`）を読み込みます。

In [35]:
# compounds.tsvを読み込み
df_substrate = pd.read_csv('dataset/compounds.tsv', sep='\t', encoding='latin-1')
# 必要な列を抽出
df_substrate = df_substrate[["CHEBI_ACCESSION", "NAME"]]
# 列名を変更
df_substrate.columns = ["ChEBI", "Substrate"]
# df_substrateを表示
df_substrate

,ChEBI,Substrate
0,CHEBI:9349,sulfonyldimethane
1,CHEBI:9352,sulindac
2,CHEBI:9355,sulfuretin
3,CHEBI:9380,syringin
4,CHEBI:9427,2-methylanthraquinone
...,...,...
228011,CHEBI:707880,NaN
228012,CHEBI:718203,NaN
228013,CHEBI:741566,NaN
228014,CHEBI:232829,NaN


### **(2) UniProtデータとChEBI IDを統合**

-   `df_prot_sub`と`df_substrate`をChEBI IDをキーにして統合し、`df_transporter`を作成します。
-   `df_transporter`には、UniProt ID、ChEBI ID、基質名（Substrate）が含まれます。

In [37]:
# df_prot_subとdf_substrateを"ChEBI"列でマージ
df_transporter = df_prot_sub.merge(df_substrate[["ChEBI", "Substrate"]], on="ChEBI", how="left")
# さらにdf_uniprotと"Uniprot ID"でマージ
df_transporter = df_transporter.merge(df_uniprot[["Entry", "Sequence", "Annotation"]], left_on="Uniprot ID", right_on="Entry", how="left")
# 不要な列を削除
df_transporter = df_transporter.drop(columns=["Entry"])
# df_transporterを表示
df_transporter

,Uniprot ID,ChEBI,Substrate,Sequence,Annotation
0,A0A060XZ13,CHEBI:167478,coproporphyrin I(4-),MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,5.0
1,A0A060XZ13,CHEBI:131725,coproporphyrin III(4-),MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,5.0
2,A0A060XZ13,CHEBI:57309,coproporphyrinogen III(4-),MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,5.0
3,A0A060XZ13,CHEBI:60344,ferroheme b(2-),MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,5.0
4,A0A060XZ13,CHEBI:58687,pheophorbide a(2-),MLLINEDERDVEGGTGQLLLDGAEESQSTWHDFGQKVRLLVPYMWP...,5.0
...,...,...,...,...,...
216944,Y0KIE2,CHEBI:15378,hydron,MLENYFPILMFILVGLGVGLGPLILGKLVSPHRPDSEKNSPYECGF...,3.0
216945,Z9JMX9,CHEBI:29103,potassium(1+),MAHASTKSLFTAELIVPAIGDAFKKLNPKELIRNPVMFVTACVALL...,3.0
216946,Z9JU65,CHEBI:15378,hydron,MNPDTITLSAGSLTTVAVIVVIALIAVAMALRFRAEVLRAATGTPA...,3.0
216947,Z9JXE3,CHEBI:15378,hydron,MAELTIRPEDIRHALDTAVSTYQAGSTDREEIGRVTESADGIARVE...,3.0


### **(3) substrate_countsの確認**

-   基質の種類ごとの出現回数をカウントし、ファイルに保存します。
-   これにより、データセット内の基質の分布を確認できます。

In [38]:
# df_transporter['Substrate'].value_counts()を保存
df_transporter['Substrate'].value_counts().to_csv('dataset/substrate_counts.csv')

---

## **6. 最終データセットの作成**

### **(1) 1行1配列に変換**

-   `df_transporter`を、UniProt IDと配列（Sequence）ごとに1行になるように変換します。
-   各行には、UniProt ID、配列、対応する基質（複数存在する場合はリスト形式）が含まれます。

In [39]:
# df_transporterのUniprot ID, Sequence, Substrateを取得
df_transporter2 = df_transporter[["Uniprot ID", "Sequence", "Substrate", "Annotation"]]
# Uniprot IDとSequenceごとにグループ化し、Substrateをリスト化
df_unique = df_transporter2.groupby(['Uniprot ID', 'Sequence', "Annotation"], as_index=False).agg({'Substrate': list})
# Substrateカラム名をSubstratesに変更
df_unique = df_unique.rename(columns={'Substrate': 'Substrates'})
# df_uniqueを表示
df_unique

,Uniprot ID,Sequence,Annotation,Substrates
0,A0A009HCH6,MSAITPYDWAIIAFVIGVTFLCVFMLTVPLLLGGKSWGRAKQEQFE...,3.0,[hydron]
1,A0A009HR43,MKYTLTRANPDADQYPLQDRQIVTDPLEEEVNKNVFMTRLEDVLHT...,3.0,[hydron]
2,A0A009IS56,MNFTVSFSTLMPLAPVMIVALTAVVVMLLISIKRNHNLIATTSVVG...,3.0,[hydron]
3,A0A009ITI1,MKGNRDVINQLNQVLYHHLTAINQYFLHSRMFNDWGIEQLGSAEYK...,3.0,[iron(2+)]
4,A0A009YQY2,MSTQIQSQQTSQQRIEAIKSAFVKLLPQHMLKNPVMGIVWLGTLIT...,3.0,[potassium(1+)]
...,...,...,...,...
107928,Z4WWK6,MEQLRSSLRDKPAARWGALALVSLTMFFAYMFVDVLSPVKILVEKE...,4.0,"[Ala-Lys(1+), L-aminoacyl-L-arginine(1+), L-am..."
107929,Z9JMX9,MAHASTKSLFTAELIVPAIGDAFKKLNPKELIRNPVMFVTACVALL...,3.0,[potassium(1+)]
107930,Z9JU65,MNPDTITLSAGSLTTVAVIVVIALIAVAMALRFRAEVLRAATGTPA...,3.0,[hydron]
107931,Z9JXE3,MAELTIRPEDIRHALDTAVSTYQAGSTDREEIGRVTESADGIARVE...,3.0,[hydron]


In [42]:
# substrates_counts.csvとして保存
df_unique['Substrates'].value_counts().to_csv('dataset/substrates_counts.csv')

### **(2) hydronを削除**

-   `hydron`のみを基質とするエントリが多すぎるため、`Annotation`が3.0のエントリを削除します。
-   これにより、データセットのバランスを調整します。

In [40]:
# 削除したい条件に合致する行のインデックスを取得
mask = (df_unique['Substrates'].apply(lambda x: x == ['hydron'])) & (df_unique['Annotation'] == 3.0)
indices_to_cut = df_unique[mask].index

# 選ばれたインデックスを除外
df_reduced = df_unique.drop(indices_to_cut)
# df_reducedを表示
df_reduced

,Uniprot ID,Sequence,Annotation,Substrates
3,A0A009ITI1,MKGNRDVINQLNQVLYHHLTAINQYFLHSRMFNDWGIEQLGSAEYK...,3.0,[iron(2+)]
4,A0A009YQY2,MSTQIQSQQTSQQRIEAIKSAFVKLLPQHMLKNPVMGIVWLGTLIT...,3.0,[potassium(1+)]
7,A0A010S454,MPDTPESPVPVQDEEKTQPDIDIQALGISPADLPPIDTEDETLQRI...,4.0,"[Ala-Lys(1+), L-aminoacyl-L-arginine(1+), L-am..."
9,A0A011AGK4,MTLIQEPPVQTPPSPPRRVANGLLDPKLLLTSLPDAVKKLDPRVMV...,3.0,[potassium(1+)]
14,A0A011NDH9,MAIRIQNVNFFYGSNQALFDINLEIEKGDTVVLLGPSGAGKSTLIR...,3.0,"[L-argininium(1+), polar amino acid zwitterion]"
...,...,...,...,...
107920,X8GRW8,MSKGKYAVTGMSCAACQARVEKAVNQLDHVDHAVVNLLTNSMTVEG...,3.0,[copper(1+)]
107922,X8GY71,MELFFLIGFIGALVALLFAQSRRLVVMACPEGNDRMRKIASAIREG...,3.0,[sodium(1+)]
107925,Y0KH45,MSSPAIKSLLDPTLVKPAIVDSFKKLAPQIQWRNPVMFVVYLGSLL...,3.0,[potassium(1+)]
107928,Z4WWK6,MEQLRSSLRDKPAARWGALALVSLTMFFAYMFVDVLSPVKILVEKE...,4.0,"[Ala-Lys(1+), L-aminoacyl-L-arginine(1+), L-am..."


### **(3) substrate_countsの確認**

-   `hydron`削除後の基質の数をカウントし、基質ごとのエントリ数を確認します。

In [41]:
# substrates_counts_after_reducing_hydron.csvとして保存
df_reduced['Substrates'].value_counts().to_csv('dataset/substrates_counts_after_reducing_hydron.csv')

---

## **7. 学習データセットの生成**

### **(1) データの分割と整形と保存**

-   データをランダムにトレーニングデータセット(90%)とテストデータセット(10％)にランダム分割し、JSON形式で保存します。
-   保存の際、以下の形式でデータを整形します。

    ```json
    {
        "instruction": "[Generate by Substrate]",
        "input": "Substrate=<substrate1>, <substrate2>, ...",
        "output": "Seq=<sequence>"
    }
    ```

    ```json
    {
        "instruction": "[Determine Substrate]",
        "input": "Seq=<sequence>",
        "output": "Substrate=<substrate1>, <substrate2>, ..."
    }
    ```

In [16]:
from sklearn.model_selection import train_test_split
import json

def save_to_json_split(sequences, substrates, test_size=0.1, random_state=0, save_dir="."):
    """
    データをトレーニング用とテスト用に分割し、JSONファイルに保存する関数。

    Args:
        sequences: タンパク質配列のリスト。
        substrates: 基質名のリストのリスト。
        test_size: テストデータの割合 (例: 0.2 なら20%)。
        random_state: ランダムシード。
        save_dir: JSONファイルを保存するディレクトリ。
    """

    # データをトレーニング用とテスト用に分割
    train_sequences, test_sequences, train_substrates, test_substrates = train_test_split(
        sequences, substrates, test_size=test_size, random_state=random_state
    )

    # トレーニングデータを作成して保存
    train_data_list = []
    for i in range(len(train_sequences)):
        substrates_string = ", ".join(train_substrates[i])
        substrates_string_formatted = f"Substrate=<{substrates_string}>"

        # 基質から配列を生成する指示
        data_point_generate = {
            "instruction": "[Generate by Substrate]",
            "input": substrates_string_formatted,
            "output": f"Seq=<{train_sequences[i]}>"
        }
        train_data_list.append(data_point_generate)

        # 配列から基質を決定する指示
        data_point_determine = {
            "instruction": "[Determine Substrate]",
            "input": f"Seq=<{train_sequences[i]}>",
            "output": substrates_string_formatted  # 整形された文字列を使用
        }
        train_data_list.append(data_point_determine)

    with open(join(save_dir, "train_split.json"), 'w') as f:
        json.dump(train_data_list, f, indent=4)

    # テストデータを作成して保存
    test_data_list = []
    for i in range(len(test_sequences)):
        substrates_string = ", ".join(test_substrates[i])
        substrates_string_formatted = f"Substrate=<{substrates_string}>"

        # 基質から配列を生成する指示
        data_point_generate = {
            "instruction": "[Generate by Substrate]",
            "input": substrates_string_formatted,
            "output": f"Seq=<{test_sequences[i]}>"
        }
        test_data_list.append(data_point_generate)

        # 配列から基質を決定する指示
        data_point_determine = {
            "instruction": "[Determine Substrate]",
            "input": f"Seq=<{test_sequences[i]}>",
            "output": substrates_string_formatted  # 整形された文字列を使用
        }
        test_data_list.append(data_point_determine)

    with open(join(save_dir, "test_split.json"), 'w') as f:
        json.dump(test_data_list, f, indent=4)

# シーケンスと基質を取得
sequences = df_reduced["Sequence"].tolist()
substrates = df_reduced["Substrates"].tolist()

# データを分割し、JSONファイルに保存
save_to_json_split(sequences, substrates, test_size=0.1, random_state=0, save_dir="dataset")

### **(2) tokenizerをロードしてparquet形式のデータに変換**

-   トレーニングとテストの各データセットに対して、json形式→parquet形式に変換

In [17]:
from build_instruction_dataset import tokenize_instruction_dataset
from transformers import AutoTokenizer

MAX_SEQ_LEN = 1024
TRAIN_DATA_PATH = "dataset/train_split.json"
TEST_DATA_PATH = "dataset/test_split.json"
tokenizer = AutoTokenizer.from_pretrained("GreatCaptainNemo/ProLLaMA_Stage_1")

train_dataset = tokenize_instruction_dataset(tokenizer=tokenizer, data_cache_dir=None, data_path=TRAIN_DATA_PATH, max_seq_length=MAX_SEQ_LEN, preprocessing_num_workers=4)
test_dataset = tokenize_instruction_dataset(tokenizer=tokenizer, data_cache_dir=None, data_path=TEST_DATA_PATH, max_seq_length=MAX_SEQ_LEN, preprocessing_num_workers=4)

print(f"train_dataset size: {len(train_dataset)}")
print(f"train longest number of tokens: {max([len(x) for x in train_dataset['input_ids']])}")
print(f"test_dataset size: {len(test_dataset)}")
print(f"test longest number of tokens: {max([len(x) for x in test_dataset['input_ids']])}")

train_dataset.to_parquet(f"dataset/train-{MAX_SEQ_LEN}.parquet")
test_dataset.to_parquet(f"dataset/test-{MAX_SEQ_LEN}.parquet")

train_dataset size: 192300
train longest number of tokens: 888
test_dataset size: 21368
test longest number of tokens: 779


Creating parquet from Arrow format: 100%|██████████| 22/22 [00:00<00:00, 39.69ba/s]


107589976